Weather ETL

![bttf image](bttflogo.png)

Data is being collected to understand what were the weather conditions during a shipment and how those conditions influence the fuel consumption. 

In order to showcase my interpretation of the solution the transformation is dissected and showing all the outputs for the specific cell.

In this case the weather data arrives into a landing zone, so in order to reach it let's glob the files:

*Globbing this data takes about 50 secs and the current storage use of the landing zone is 2.26 MB



In [1]:
import glob
import os
import json
import pandas as pd



# getting the json files names from the landing zone and appending into the data dict
globbed_files = glob.glob(r"C:/Users/E.ALVAREZHERNANDEZ/VS Projects/caseStudy/landing_zone/weather/*.json")

data = [] 
for file in globbed_files:
    frame = pd.read_json(file, lines=True)
    data.append(frame)

data


[        lat       lon      city  \
 0  47.49835  19.04045  Budapest   
 
                                              weather  
 0  {'dt': 1656637200, 'temp': -11, 'wind': [{'win...  ,
        lat      lon    city                                            weather
 0  40.4165 -3.70256  Madrid  {'dt': 1656637200, 'temp': 46, 'wind': [{'wind...,
     lat       lon   city                                            weather
 0  53.9  27.56667  Minsk  {'dt': 1656637200, 'temp': 5, 'wind': [{'wind_...,
         lat       lon    city  \
 0  55.75222  37.61556  Moscow   
 
                                              weather  
 0  {'dt': 1656637200, 'temp': 45, 'wind': [{'wind...  ,
         lat       lon    city  \
 0  52.22977  21.01178  Warsaw   
 
                                              weather  
 0  {'dt': 1656637200, 'temp': 22, 'wind': [{'wind...  ,
         lat       lon    city  \
 0  39.91987  32.85427  Ankara   
 
                                              weather  
 0  {

In [2]:
#Concatenating everything into a single dataframe

df = pd.concat(data, ignore_index=True)

df

,lat,lon,city,weather
0,47.49835,19.04045,Budapest,"{'dt': 1656637200, 'temp': -11, 'wind': [{'win..."
1,40.41650,-3.70256,Madrid,"{'dt': 1656637200, 'temp': 46, 'wind': [{'wind..."
2,53.90000,27.56667,Minsk,"{'dt': 1656637200, 'temp': 5, 'wind': [{'wind_..."
3,55.75222,37.61556,Moscow,"{'dt': 1656637200, 'temp': 45, 'wind': [{'wind..."
4,52.22977,21.01178,Warsaw,"{'dt': 1656637200, 'temp': 22, 'wind': [{'wind..."
...,...,...,...,...
10135,51.50853,-0.12574,London,"{'dt': 1659384000, 'temp': -13, 'wind': [{'win..."
10136,40.41650,-3.70256,Madrid,"{'dt': 1659384000, 'temp': -2, 'wind': [{'wind..."
10137,40.41650,-3.70256,Madrid,"{'dt': 1659394800, 'temp': -27, 'wind': [{'win..."
10138,55.75222,37.61556,Moscow,"{'dt': 1659402000, 'temp': 37, 'wind': [{'wind..."


1. Flattening: 

Our first challenge is to flatten the data, since relevant values are in a struct form.

In [3]:
#First I normalize the first level of the wind column
from pandas import json_normalize


df = df.join(json_normalize(df['weather'].to_list()))\
       .drop(['weather'], axis=1)

df

,lat,lon,city,dt,temp,wind,clouds,humidity,pressure,feels_like,visibility
0,47.49835,19.04045,Budapest,1656637200,-11,"[{'wind_deg': -9.9, 'wind_speed': 95}]",6,23,1013,-12.1,10000
1,40.41650,-3.70256,Madrid,1656637200,46,"[{'wind_deg': 41.4, 'wind_speed': 17}]",100,15,1019,50.6,10000
2,53.90000,27.56667,Minsk,1656637200,5,"[{'wind_deg': 4.5, 'wind_speed': 74}]",47,27,1009,5.5,10000
3,55.75222,37.61556,Moscow,1656637200,45,"[{'wind_deg': 40.5, 'wind_speed': 68}]",51,24,1015,49.5,10000
4,52.22977,21.01178,Warsaw,1656637200,22,"[{'wind_deg': 19.8, 'wind_speed': 25}]",92,13,1007,24.2,10000
...,...,...,...,...,...,...,...,...,...,...,...
10135,51.50853,-0.12574,London,1659384000,-13,"[{'wind_deg': -11.7, 'wind_speed': 95}]",51,15,1005,-14.3,10000
10136,40.41650,-3.70256,Madrid,1659384000,-2,"[{'wind_deg': -1.8, 'wind_speed': 98}]",43,22,1019,-2.2,10000
10137,40.41650,-3.70256,Madrid,1659394800,-27,"[{'wind_deg': -24.3, 'wind_speed': 29}]",88,18,1007,-29.7,10000
10138,55.75222,37.61556,Moscow,1659402000,37,"[{'wind_deg': 33.3, 'wind_speed': 75}]",65,13,1016,40.7,10000


In [4]:
#Normalize to enter into the second level
df = df.join(json_normalize(df['wind'].to_list()))\
       .drop(['wind'], axis=1)

df

,lat,lon,city,dt,temp,clouds,humidity,pressure,feels_like,visibility,0
0,47.49835,19.04045,Budapest,1656637200,-11,6,23,1013,-12.1,10000,"{'wind_deg': -9.9, 'wind_speed': 95}"
1,40.41650,-3.70256,Madrid,1656637200,46,100,15,1019,50.6,10000,"{'wind_deg': 41.4, 'wind_speed': 17}"
2,53.90000,27.56667,Minsk,1656637200,5,47,27,1009,5.5,10000,"{'wind_deg': 4.5, 'wind_speed': 74}"
3,55.75222,37.61556,Moscow,1656637200,45,51,24,1015,49.5,10000,"{'wind_deg': 40.5, 'wind_speed': 68}"
4,52.22977,21.01178,Warsaw,1656637200,22,92,13,1007,24.2,10000,"{'wind_deg': 19.8, 'wind_speed': 25}"
...,...,...,...,...,...,...,...,...,...,...,...
10135,51.50853,-0.12574,London,1659384000,-13,51,15,1005,-14.3,10000,"{'wind_deg': -11.7, 'wind_speed': 95}"
10136,40.41650,-3.70256,Madrid,1659384000,-2,43,22,1019,-2.2,10000,"{'wind_deg': -1.8, 'wind_speed': 98}"
10137,40.41650,-3.70256,Madrid,1659394800,-27,88,18,1007,-29.7,10000,"{'wind_deg': -24.3, 'wind_speed': 29}"
10138,55.75222,37.61556,Moscow,1659402000,37,65,13,1016,40.7,10000,"{'wind_deg': 33.3, 'wind_speed': 75}"


In [5]:
#Getting into the wind information
df = df.join(json_normalize(df[0].to_list()))\
       .drop([0], axis=1)

df

,lat,lon,city,dt,temp,clouds,humidity,pressure,feels_like,visibility,wind_deg,wind_speed
0,47.49835,19.04045,Budapest,1656637200,-11,6,23,1013,-12.1,10000,-9.9,95
1,40.41650,-3.70256,Madrid,1656637200,46,100,15,1019,50.6,10000,41.4,17
2,53.90000,27.56667,Minsk,1656637200,5,47,27,1009,5.5,10000,4.5,74
3,55.75222,37.61556,Moscow,1656637200,45,51,24,1015,49.5,10000,40.5,68
4,52.22977,21.01178,Warsaw,1656637200,22,92,13,1007,24.2,10000,19.8,25
...,...,...,...,...,...,...,...,...,...,...,...,...
10135,51.50853,-0.12574,London,1659384000,-13,51,15,1005,-14.3,10000,-11.7,95
10136,40.41650,-3.70256,Madrid,1659384000,-2,43,22,1019,-2.2,10000,-1.8,98
10137,40.41650,-3.70256,Madrid,1659394800,-27,88,18,1007,-29.7,10000,-24.3,29
10138,55.75222,37.61556,Moscow,1659402000,37,65,13,1016,40.7,10000,33.3,75


2. DateTime Format:

 Datetime data is in an Epoch format, in order to be human readable we need to transform it into a datetime format.

In [6]:
#Time was in Epoch format so we need to change it into datetime

import datetime

df['dt'] = pd.to_datetime(df['dt'], unit='s')

df


,lat,lon,city,dt,temp,clouds,humidity,pressure,feels_like,visibility,wind_deg,wind_speed
0,47.49835,19.04045,Budapest,2022-07-01 01:00:00,-11,6,23,1013,-12.1,10000,-9.9,95
1,40.41650,-3.70256,Madrid,2022-07-01 01:00:00,46,100,15,1019,50.6,10000,41.4,17
2,53.90000,27.56667,Minsk,2022-07-01 01:00:00,5,47,27,1009,5.5,10000,4.5,74
3,55.75222,37.61556,Moscow,2022-07-01 01:00:00,45,51,24,1015,49.5,10000,40.5,68
4,52.22977,21.01178,Warsaw,2022-07-01 01:00:00,22,92,13,1007,24.2,10000,19.8,25
...,...,...,...,...,...,...,...,...,...,...,...,...
10135,51.50853,-0.12574,London,2022-08-01 20:00:00,-13,51,15,1005,-14.3,10000,-11.7,95
10136,40.41650,-3.70256,Madrid,2022-08-01 20:00:00,-2,43,22,1019,-2.2,10000,-1.8,98
10137,40.41650,-3.70256,Madrid,2022-08-01 23:00:00,-27,88,18,1007,-29.7,10000,-24.3,29
10138,55.75222,37.61556,Moscow,2022-08-02 01:00:00,37,65,13,1016,40.7,10000,33.3,75


3. Sinking weather into Data Lake:

The sink is being performed into parquet format and compressed in snappy in order to optimize performance and the storage use

In [7]:
#Sinking the clean data into the datalake, partitioned by city

df.to_parquet('C:/Users/E.ALVAREZHERNANDEZ/VS Projects/caseStudy/clean_data/weather/',compression='snappy', partition_cols='city')

4. Delta weather table:

As we recreate the data from the datalake parquet files the processing times improves, from 56 secs to 0.1 secs and the storage use from 2.26 MB to 320 KB, it reduces the size by 84%

In [52]:
#Pulling clean weather temperature

cweather = pd.read_parquet('C:/Users/E.ALVAREZHERNANDEZ/VS Projects/caseStudy/clean_data/weather')

cweather

,lat,lon,dt,temp,clouds,humidity,pressure,feels_like,visibility,wind_deg,wind_speed,city
4508,47.49835,19.04045,2022-07-01 01:00:00,-11,6,23,1013,-12.1,10000,-9.9,95,Budapest
4509,47.49835,19.04045,2022-07-01 03:00:00,-14,12,30,1019,-15.4,10000,-12.6,35,Budapest
4510,47.49835,19.04045,2022-07-01 05:00:00,20,91,11,1006,22.0,10000,18.0,86,Budapest
4511,47.49835,19.04045,2022-07-01 08:00:00,-9,33,29,1006,-9.9,10000,-8.1,15,Budapest
4512,47.49835,19.04045,2022-07-01 09:00:00,34,50,24,1007,37.4,10000,30.6,48,Budapest
...,...,...,...,...,...,...,...,...,...,...,...,...
5647,47.49835,19.04045,2022-07-31 18:00:00,8,57,22,1017,8.8,10000,7.2,77,Budapest
5648,47.49835,19.04045,2022-07-31 19:00:00,32,49,26,1015,35.2,10000,28.8,63,Budapest
5649,47.49835,19.04045,2022-07-31 21:00:00,40,73,19,1017,44.0,10000,36.0,87,Budapest
5650,47.49835,19.04045,2022-07-31 22:00:00,13,14,24,1011,14.3,10000,11.7,20,Budapest


In [9]:
#In order to get the avg temperature I will subset

temperature = cweather[['dt', 'temp', 'city']]

temperature

,dt,temp,city
0,2022-07-01 02:00:00,48,Ankara
1,2022-07-01 03:00:00,26,Ankara
2,2022-07-01 05:00:00,-26,Ankara
3,2022-07-01 06:00:00,10,Ankara
4,2022-07-01 10:00:00,-18,Ankara
...,...,...,...
20275,2022-07-31 11:00:00,37,Warsaw
20276,2022-07-31 12:00:00,24,Warsaw
20277,2022-07-31 14:00:00,-6,Warsaw
20278,2022-07-31 16:00:00,-5,Warsaw


5. Connecting with shipments DB:


In [10]:
import psycopg2
import logging

conn = psycopg2.connect(host='localhost',
                        database='bttf',
                        user='postgres',
                        password='Zurich2022!',
                        port='5432')



In [11]:
pointer = conn.cursor()

pointer.execute("SELECT * FROM shipments.shipments")

rows = pointer.fetchall() #Tupple format


In [12]:
shdf = pd.DataFrame(rows, columns=['id', 'truck', 'driver', 'shipment_start_timestamp', 'shipment_end_timestamp', 'start_location', 'end_location', 'shipment_distance', 'consumed_fuel'])
shdf

,id,truck,driver,shipment_start_timestamp,shipment_end_timestamp,start_location,end_location,shipment_distance,consumed_fuel
0,9919,6305,4272,2022-07-29 20:12:53.027776,2022-07-30 22:52:32.903400,Hamburg,Barcelona,1769.40660,342.89
1,9963,6931,3438,2022-07-23 19:33:42.412344,2022-07-25 07:43:57.066507,Hamburg,Ankara,2752.17850,506.77
2,9969,2514,4797,2022-07-29 18:01:08.431993,2022-07-30 06:24:11.660956,Hamburg,London,865.48140,161.01
3,9913,2531,9127,2022-07-06 00:16:40.813996,2022-07-06 14:22:57.521956,Hamburg,Stockholm,974.03534,180.04
4,9691,9035,7504,2022-07-18 01:01:13.147528,2022-07-18 17:31:10.597189,Hamburg,Budapest,1113.69500,204.33
...,...,...,...,...,...,...,...,...,...
9440,1193,9871,3150,2022-07-28 11:45:55.328784,2022-07-28 20:12:33.632634,Vienna,Warsaw,667.40424,111.79
9441,875,9443,1055,2022-07-10 20:16:38.103979,2022-07-11 04:53:11.526090,Vienna,Warsaw,667.40424,117.95
9442,462,6301,2606,2022-07-19 10:22:46.344214,2022-07-19 19:27:31.316003,Vienna,Warsaw,667.40424,118.04
9443,388,8058,4739,2022-07-03 20:23:03.033107,2022-07-04 05:38:58.188670,Vienna,Warsaw,667.40424,115.37


6. Fixing shipments timestamp formats:

Asumption: Weather doesn't have a big variance from hour to hour, so to facilitate the analysis I'm going to round the shipment hours to the closest hour.

In [13]:
#Dates are in sql format, so in here I invert them to match the df convention

import datetime


shdf['shipment_start_timestamp'] = pd.to_datetime(shdf['shipment_start_timestamp'] ).apply(lambda x: datetime.datetime.strftime(x, '%Y-%m-%d %H:%M:%S'))
shdf['shipment_end_timestamp'] = pd.to_datetime(shdf['shipment_end_timestamp'] ).apply(lambda x: datetime.datetime.strftime(x, '%Y-%m-%d %H:%M:%S'))
shdf['shipment_start_timestamp'] = pd.to_datetime(shdf['shipment_start_timestamp'])
shdf['shipment_end_timestamp'] = pd.to_datetime(shdf['shipment_end_timestamp'])



shdf

,id,truck,driver,shipment_start_timestamp,shipment_end_timestamp,start_location,end_location,shipment_distance,consumed_fuel
0,9919,6305,4272,2022-07-29 20:12:53,2022-07-30 22:52:32,Hamburg,Barcelona,1769.40660,342.89
1,9963,6931,3438,2022-07-23 19:33:42,2022-07-25 07:43:57,Hamburg,Ankara,2752.17850,506.77
2,9969,2514,4797,2022-07-29 18:01:08,2022-07-30 06:24:11,Hamburg,London,865.48140,161.01
3,9913,2531,9127,2022-07-06 00:16:40,2022-07-06 14:22:57,Hamburg,Stockholm,974.03534,180.04
4,9691,9035,7504,2022-07-18 01:01:13,2022-07-18 17:31:10,Hamburg,Budapest,1113.69500,204.33
...,...,...,...,...,...,...,...,...,...
9440,1193,9871,3150,2022-07-28 11:45:55,2022-07-28 20:12:33,Vienna,Warsaw,667.40424,111.79
9441,875,9443,1055,2022-07-10 20:16:38,2022-07-11 04:53:11,Vienna,Warsaw,667.40424,117.95
9442,462,6301,2606,2022-07-19 10:22:46,2022-07-19 19:27:31,Vienna,Warsaw,667.40424,118.04
9443,388,8058,4739,2022-07-03 20:23:03,2022-07-04 05:38:58,Vienna,Warsaw,667.40424,115.37


In [14]:
shdf['shipment_start_timestamp'] = shdf['shipment_start_timestamp'].dt.round('H')
shdf['shipment_end_timestamp'] = shdf['shipment_end_timestamp'].dt.round('H')
shdf['fConsumption']= (shdf['consumed_fuel']*100)/shdf['shipment_distance']
shdf


,id,truck,driver,shipment_start_timestamp,shipment_end_timestamp,start_location,end_location,shipment_distance,consumed_fuel,fConsumption
0,9919,6305,4272,2022-07-29 20:00:00,2022-07-30 23:00:00,Hamburg,Barcelona,1769.40660,342.89,19.378813
1,9963,6931,3438,2022-07-23 20:00:00,2022-07-25 08:00:00,Hamburg,Ankara,2752.17850,506.77,18.413413
2,9969,2514,4797,2022-07-29 18:00:00,2022-07-30 06:00:00,Hamburg,London,865.48140,161.01,18.603519
3,9913,2531,9127,2022-07-06 00:00:00,2022-07-06 14:00:00,Hamburg,Stockholm,974.03534,180.04,18.483929
4,9691,9035,7504,2022-07-18 01:00:00,2022-07-18 18:00:00,Hamburg,Budapest,1113.69500,204.33,18.347034
...,...,...,...,...,...,...,...,...,...,...
9440,1193,9871,3150,2022-07-28 12:00:00,2022-07-28 20:00:00,Vienna,Warsaw,667.40424,111.79,16.749969
9441,875,9443,1055,2022-07-10 20:00:00,2022-07-11 05:00:00,Vienna,Warsaw,667.40424,117.95,17.672947
9442,462,6301,2606,2022-07-19 10:00:00,2022-07-19 19:00:00,Vienna,Warsaw,667.40424,118.04,17.686432
9443,388,8058,4739,2022-07-03 20:00:00,2022-07-04 06:00:00,Vienna,Warsaw,667.40424,115.37,17.286375


7. Merging data:

In this step we merge the dataframes to get the weather conditions at location and time for the shipments

In [15]:
#Merging to get the first temperature matching the start location and the start timestamp


dfmerge = pd.merge(shdf, temperature, how='left', left_on=[shdf['start_location'].astype(str), shdf['shipment_start_timestamp'].astype(str)], right_on=[temperature['city'].astype(str), temperature['dt'].astype(str)])
dfmerge= dfmerge.rename(columns={'temp':'startTemp'}).drop(['key_0', 'key_1', 'city', 'dt'], axis=1)

dfmerge

,id,truck,driver,shipment_start_timestamp,shipment_end_timestamp,start_location,end_location,shipment_distance,consumed_fuel,fConsumption,startTemp
0,9919,6305,4272,2022-07-29 20:00:00,2022-07-30 23:00:00,Hamburg,Barcelona,1769.40660,342.89,19.378813,3.0
1,9919,6305,4272,2022-07-29 20:00:00,2022-07-30 23:00:00,Hamburg,Barcelona,1769.40660,342.89,19.378813,3.0
2,9963,6931,3438,2022-07-23 20:00:00,2022-07-25 08:00:00,Hamburg,Ankara,2752.17850,506.77,18.413413,NaN
3,9969,2514,4797,2022-07-29 18:00:00,2022-07-30 06:00:00,Hamburg,London,865.48140,161.01,18.603519,23.0
4,9969,2514,4797,2022-07-29 18:00:00,2022-07-30 06:00:00,Hamburg,London,865.48140,161.01,18.603519,23.0
...,...,...,...,...,...,...,...,...,...,...,...
17768,462,6301,2606,2022-07-19 10:00:00,2022-07-19 19:00:00,Vienna,Warsaw,667.40424,118.04,17.686432,-17.0
17769,388,8058,4739,2022-07-03 20:00:00,2022-07-04 06:00:00,Vienna,Warsaw,667.40424,115.37,17.286375,45.0
17770,388,8058,4739,2022-07-03 20:00:00,2022-07-04 06:00:00,Vienna,Warsaw,667.40424,115.37,17.286375,45.0
17771,121,8171,5326,2022-07-28 22:00:00,2022-07-29 07:00:00,Vienna,Warsaw,667.40424,114.60,17.171003,29.0


In [16]:
#Merging to get the end temperature matching the end location and the end timestamp


dfmerge2 = pd.merge(dfmerge, temperature, how='left', left_on=[dfmerge['end_location'].astype(str), dfmerge['shipment_end_timestamp'].astype(str)], right_on=[temperature['city'].astype(str), temperature['dt'].astype(str)])
dfmerge2= dfmerge2.drop(['key_0', 'key_1', 'dt', 'city'], axis=1).rename(columns={'temp':'endTemp'})
dfmerge2

,id,truck,driver,shipment_start_timestamp,shipment_end_timestamp,start_location,end_location,shipment_distance,consumed_fuel,fConsumption,startTemp,endTemp
0,9919,6305,4272,2022-07-29 20:00:00,2022-07-30 23:00:00,Hamburg,Barcelona,1769.40660,342.89,19.378813,3.0,NaN
1,9919,6305,4272,2022-07-29 20:00:00,2022-07-30 23:00:00,Hamburg,Barcelona,1769.40660,342.89,19.378813,3.0,NaN
2,9963,6931,3438,2022-07-23 20:00:00,2022-07-25 08:00:00,Hamburg,Ankara,2752.17850,506.77,18.413413,NaN,-2.0
3,9963,6931,3438,2022-07-23 20:00:00,2022-07-25 08:00:00,Hamburg,Ankara,2752.17850,506.77,18.413413,NaN,-2.0
4,9969,2514,4797,2022-07-29 18:00:00,2022-07-30 06:00:00,Hamburg,London,865.48140,161.01,18.603519,23.0,-14.0
...,...,...,...,...,...,...,...,...,...,...,...,...
33503,388,8058,4739,2022-07-03 20:00:00,2022-07-04 06:00:00,Vienna,Warsaw,667.40424,115.37,17.286375,45.0,15.0
33504,121,8171,5326,2022-07-28 22:00:00,2022-07-29 07:00:00,Vienna,Warsaw,667.40424,114.60,17.171003,29.0,-20.0
33505,121,8171,5326,2022-07-28 22:00:00,2022-07-29 07:00:00,Vienna,Warsaw,667.40424,114.60,17.171003,29.0,-20.0
33506,121,8171,5326,2022-07-28 22:00:00,2022-07-29 07:00:00,Vienna,Warsaw,667.40424,114.60,17.171003,29.0,-20.0


In [17]:
#Getting the average temperature for the trip, and setting the timestamp to datetime as it was required to pass them as strings for the merge

shipment = dfmerge2

shipment['avgTemp']= (shipment['startTemp']+shipment['endTemp'])/2

shipment['shipment_start_timestamp'] = pd.to_datetime(shipment['shipment_start_timestamp'])
shipment['shipment_end_timestamp'] = pd.to_datetime(shipment['shipment_end_timestamp'])

shipment['duration'] = (shipment['shipment_end_timestamp'] - shipment['shipment_start_timestamp']).astype('timedelta64[h]')

shipment

,id,truck,driver,shipment_start_timestamp,shipment_end_timestamp,start_location,end_location,shipment_distance,consumed_fuel,fConsumption,startTemp,endTemp,avgTemp,duration
0,9919,6305,4272,2022-07-29 20:00:00,2022-07-30 23:00:00,Hamburg,Barcelona,1769.40660,342.89,19.378813,3.0,NaN,NaN,27.0
1,9919,6305,4272,2022-07-29 20:00:00,2022-07-30 23:00:00,Hamburg,Barcelona,1769.40660,342.89,19.378813,3.0,NaN,NaN,27.0
2,9963,6931,3438,2022-07-23 20:00:00,2022-07-25 08:00:00,Hamburg,Ankara,2752.17850,506.77,18.413413,NaN,-2.0,NaN,36.0
3,9963,6931,3438,2022-07-23 20:00:00,2022-07-25 08:00:00,Hamburg,Ankara,2752.17850,506.77,18.413413,NaN,-2.0,NaN,36.0
4,9969,2514,4797,2022-07-29 18:00:00,2022-07-30 06:00:00,Hamburg,London,865.48140,161.01,18.603519,23.0,-14.0,4.5,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33503,388,8058,4739,2022-07-03 20:00:00,2022-07-04 06:00:00,Vienna,Warsaw,667.40424,115.37,17.286375,45.0,15.0,30.0,10.0
33504,121,8171,5326,2022-07-28 22:00:00,2022-07-29 07:00:00,Vienna,Warsaw,667.40424,114.60,17.171003,29.0,-20.0,4.5,9.0
33505,121,8171,5326,2022-07-28 22:00:00,2022-07-29 07:00:00,Vienna,Warsaw,667.40424,114.60,17.171003,29.0,-20.0,4.5,9.0
33506,121,8171,5326,2022-07-28 22:00:00,2022-07-29 07:00:00,Vienna,Warsaw,667.40424,114.60,17.171003,29.0,-20.0,4.5,9.0


In [18]:
#The correlation matrix shows some results already


shipment.corr()

,id,truck,driver,shipment_distance,consumed_fuel,fConsumption,startTemp,endTemp,avgTemp,duration
id,1.000000,-0.024621,0.008833,-0.002869,-0.002004,0.002043,-0.010586,0.010457,0.000072,-0.001907
truck,-0.024621,1.000000,-0.003559,-0.001682,0.004865,0.021662,0.008407,-0.006044,-0.002425,-0.001353
driver,0.008833,-0.003559,1.000000,-0.018203,-0.011565,0.020798,-0.015152,-0.016788,-0.024272,-0.017358
shipment_distance,-0.002869,-0.001682,-0.018203,1.000000,0.942831,-0.133585,-0.016421,0.003567,-0.011248,0.991306
consumed_fuel,-0.002004,0.004865,-0.011565,0.942831,1.000000,0.176680,-0.020436,0.005010,-0.013316,0.934030
fConsumption,0.002043,0.021662,0.020798,-0.133585,0.176680,1.000000,-0.011944,0.007722,-0.003363,-0.134163
startTemp,-0.010586,0.008407,-0.015152,-0.016421,-0.020436,-0.011944,1.000000,-0.001402,0.705573,-0.015851
endTemp,0.010457,-0.006044,-0.016788,0.003567,0.005010,0.007722,-0.001402,1.000000,0.707647,0.004325
avgTemp,0.000072,-0.002425,-0.024272,-0.011248,-0.013316,-0.003363,0.705573,0.707647,1.000000,-0.010078
duration,-0.001907,-0.001353,-0.017358,0.991306,0.934030,-0.134163,-0.015851,0.004325,-0.010078,1.000000


In [19]:
#Sinking the clean data into the datalake


shipment.to_parquet('C:/Users/E.ALVAREZHERNANDEZ/VS Projects/caseStudy/clean_data/DataLake/gold.parquet',compression='snappy')

In [53]:


temperature['dt'].max()


Timestamp('2022-08-02 08:00:00')

In [148]:
import requests
import json



#barcelona = 41.38879	2.15899
#budapest = 47.49835	19.04045
apiKey= 'ffca9f02-39a7-11ed-93b0-0242ac130002-ffca9f70-39a7-11ed-93b0-0242ac130002' 
response = requests.get(
  'https://api.stormglass.io/v2/weather/point',
  params={
    'lat': 41.38879,
    'lng': 2.15899,
    'params': ','.join(['airTemperature']),
    'start': '2022-07-01 01:00:00',  
    'end': '2022-08-02 08:00:00'  
  },
  headers={
    'Authorization': apiKey
  }
)



In [149]:
# Do something with response data.
json_data = response.json()



barcelona = pd.DataFrame([json_data])


barcelona= barcelona.join(json_normalize(barcelona['hours'].to_list()))

barcelona = barcelona.transpose()

barcelona = barcelona[2:]

barcelona = barcelona.join(json_normalize(barcelona[0].to_list()))\
                      .drop([0], axis=1)[['time', 'airTemperature.dwd']].rename(columns={'airTemperature.dwd':'temp'})

barcelona['city'] = 'Barcelona'

barcelona


,time,temp,city
0,2022-07-01T01:00:00+00:00,17.64,Barcelona
1,2022-07-01T02:00:00+00:00,17.71,Barcelona
2,2022-07-01T03:00:00+00:00,17.85,Barcelona
3,2022-07-01T04:00:00+00:00,17.97,Barcelona
4,2022-07-01T05:00:00+00:00,18.22,Barcelona
...,...,...,...
235,2022-07-10T20:00:00+00:00,24.40,Barcelona
236,2022-07-10T21:00:00+00:00,22.75,Barcelona
237,2022-07-10T22:00:00+00:00,21.59,Barcelona
238,2022-07-10T23:00:00+00:00,21.16,Barcelona


In [150]:
import requests
import json



#barcelona = 41.38879	2.15899
#budapest = 47.49835	19.04045
apiKey= 'ffca9f02-39a7-11ed-93b0-0242ac130002-ffca9f70-39a7-11ed-93b0-0242ac130002' 
response = requests.get(
  'https://api.stormglass.io/v2/weather/point',
  params={
    'lat': 47.4979,
    'lng': 19.0402,
    'params': ','.join(['airTemperature']),
    'start': '2022-07-01 01:00:00',  
    'end': '2022-08-02 08:00:00'  
  },
  headers={
    'Authorization': apiKey
  }
)

json_data = response.json()


json_data

{'hours': [{'airTemperature': {'noaa': 24.66, 'sg': 24.66},
   'time': '2022-07-01T01:00:00+00:00'},
  {'airTemperature': {'noaa': 24.26, 'sg': 24.26},
   'time': '2022-07-01T02:00:00+00:00'},
  {'airTemperature': {'noaa': 23.87, 'sg': 23.87},
   'time': '2022-07-01T03:00:00+00:00'},
  {'airTemperature': {'noaa': 25.15, 'sg': 25.15},
   'time': '2022-07-01T04:00:00+00:00'},
  {'airTemperature': {'noaa': 26.43, 'sg': 26.43},
   'time': '2022-07-01T05:00:00+00:00'},
  {'airTemperature': {'noaa': 27.71, 'sg': 27.71},
   'time': '2022-07-01T06:00:00+00:00'},
  {'airTemperature': {'noaa': 29.47, 'sg': 29.47},
   'time': '2022-07-01T07:00:00+00:00'},
  {'airTemperature': {'noaa': 31.24, 'sg': 31.24},
   'time': '2022-07-01T08:00:00+00:00'},
  {'airTemperature': {'noaa': 33.01, 'sg': 33.01},
   'time': '2022-07-01T09:00:00+00:00'},
  {'airTemperature': {'noaa': 34.09, 'sg': 34.09},
   'time': '2022-07-01T10:00:00+00:00'},
  {'airTemperature': {'noaa': 35.17, 'sg': 35.17},
   'time': '2022-07-

In [151]:
budapest = pd.DataFrame([json_data])


budapest= budapest.join(json_normalize(budapest['hours'].to_list()))

budapest = budapest.transpose()

budapest = budapest[2:]

budapest = budapest.join(json_normalize(budapest[0].to_list()))\
                      .drop([0], axis=1)[['time', 'airTemperature.noaa']].rename(columns={'airTemperature.noaa':'temp'})

budapest['city'] = 'Budapest'




budapest

,time,temp,city
0,2022-07-01T01:00:00+00:00,24.66,Budapest
1,2022-07-01T02:00:00+00:00,24.26,Budapest
2,2022-07-01T03:00:00+00:00,23.87,Budapest
3,2022-07-01T04:00:00+00:00,25.15,Budapest
4,2022-07-01T05:00:00+00:00,26.43,Budapest
...,...,...,...
235,2022-07-10T20:00:00+00:00,19.49,Budapest
236,2022-07-10T21:00:00+00:00,18.20,Budapest
237,2022-07-10T22:00:00+00:00,17.60,Budapest
238,2022-07-10T23:00:00+00:00,17.01,Budapest


In [152]:
apiTemperature = pd.concat([barcelona,budapest])

apiTemperature.to_parquet('C:/Users/E.ALVAREZHERNANDEZ/VS Projects/caseStudy/clean_data/DataLake/apiWeather.parquet',compression='snappy')

In [153]:
apiweather = pd.read_parquet('C:/Users/E.ALVAREZHERNANDEZ/VS Projects/caseStudy/clean_data/DataLake/apiWeather.parquet')

apiweather['time'] = pd.to_datetime(apiweather['time']).apply(lambda x: datetime.datetime.strftime(x, '%Y-%m-%d %H:%M:%S'))

apiweather

,time,temp,city
0,2022-07-01 01:00:00,17.64,Barcelona
1,2022-07-01 02:00:00,17.71,Barcelona
2,2022-07-01 03:00:00,17.85,Barcelona
3,2022-07-01 04:00:00,17.97,Barcelona
4,2022-07-01 05:00:00,18.22,Barcelona
...,...,...,...
235,2022-07-10 20:00:00,19.49,Budapest
236,2022-07-10 21:00:00,18.20,Budapest
237,2022-07-10 22:00:00,17.60,Budapest
238,2022-07-10 23:00:00,17.01,Budapest


In [163]:
newshdf = shdf[(shdf['start_location']=='Barcelona')&(shdf['end_location']=='Budapest') ]
newshdf

,id,truck,driver,shipment_start_timestamp,shipment_end_timestamp,start_location,end_location,shipment_distance,consumed_fuel,fConsumption
1666,9932,4587,2749,2022-07-04 12:00:00,2022-07-05 12:00:00,Barcelona,Budapest,1799.821,249.90,13.884714
1667,9253,9408,5519,2022-07-02 12:00:00,2022-07-03 13:00:00,Barcelona,Budapest,1799.821,260.91,14.496442
1668,9213,1036,6421,2022-07-04 10:00:00,2022-07-05 10:00:00,Barcelona,Budapest,1799.821,253.04,14.059176
1669,9144,4119,6731,2022-07-24 17:00:00,2022-07-25 19:00:00,Barcelona,Budapest,1799.821,275.44,15.303744
1670,8637,1912,8778,2022-07-12 19:00:00,2022-07-13 20:00:00,Barcelona,Budapest,1799.821,268.84,14.937041
1671,8426,3900,2238,2022-07-15 18:00:00,2022-07-16 17:00:00,Barcelona,Budapest,1799.821,263.21,14.624232
1672,8210,4137,3095,2022-07-01 23:00:00,2022-07-03 00:00:00,Barcelona,Budapest,1799.821,255.85,14.215303
1673,6649,2239,3261,2022-07-04 03:00:00,2022-07-05 06:00:00,Barcelona,Budapest,1799.821,269.68,14.983712
1674,6357,9736,3223,2022-07-02 08:00:00,2022-07-03 11:00:00,Barcelona,Budapest,1799.821,268.41,14.913150
1675,5889,7224,8588,2022-07-28 22:00:00,2022-07-29 22:00:00,Barcelona,Budapest,1799.821,249.64,13.870268


In [172]:
dfmerge3 = pd.merge(newshdf, apiweather, how='left', left_on=[newshdf['start_location'].astype(str), newshdf['shipment_start_timestamp'].astype(str)], right_on=[apiweather['city'].astype(str), apiweather['time'].astype(str)])
dfmerge3= dfmerge3.rename(columns={'temp':'startTemp'}).drop(['key_0', 'key_1', 'city', 'time'], axis=1)

dfmerge4 = pd.merge(dfmerge3, apiweather, how='left', left_on=[dfmerge3['end_location'].astype(str), dfmerge3['shipment_end_timestamp'].astype(str)], right_on=[apiweather['city'].astype(str), apiweather['time'].astype(str)])
dfmerge4= dfmerge4.drop(['key_0', 'key_1', 'time', 'city'], axis=1).rename(columns={'temp':'endTemp'})

dfmerge4 = dfmerge4.dropna(subset=['startTemp'])

dfmerge4

,id,truck,driver,shipment_start_timestamp,shipment_end_timestamp,start_location,end_location,shipment_distance,consumed_fuel,fConsumption,startTemp,endTemp
0,9932,4587,2749,2022-07-04 12:00:00,2022-07-05 12:00:00,Barcelona,Budapest,1799.821,249.90,13.884714,30.40,28.75
1,9253,9408,5519,2022-07-02 12:00:00,2022-07-03 13:00:00,Barcelona,Budapest,1799.821,260.91,14.496442,28.19,32.50
2,9213,1036,6421,2022-07-04 10:00:00,2022-07-05 10:00:00,Barcelona,Budapest,1799.821,253.04,14.059176,31.98,27.77
6,8210,4137,3095,2022-07-01 23:00:00,2022-07-03 00:00:00,Barcelona,Budapest,1799.821,255.85,14.215303,16.73,21.62
7,6649,2239,3261,2022-07-04 03:00:00,2022-07-05 06:00:00,Barcelona,Budapest,1799.821,269.68,14.983712,19.32,21.51
8,6357,9736,3223,2022-07-02 08:00:00,2022-07-03 11:00:00,Barcelona,Budapest,1799.821,268.41,14.913150,24.72,31.13
12,5582,2735,4869,2022-07-05 03:00:00,2022-07-06 03:00:00,Barcelona,Budapest,1799.821,274.09,15.228737,19.95,18.29
14,4992,8800,7752,2022-07-01 09:00:00,2022-07-02 08:00:00,Barcelona,Budapest,1799.821,269.74,14.987046,24.24,24.25
18,3670,4885,8182,2022-07-08 04:00:00,2022-07-09 03:00:00,Barcelona,Budapest,1799.821,254.16,14.121404,18.44,14.85


In [173]:
dfmerge4.corr()

,id,truck,driver,shipment_distance,consumed_fuel,fConsumption,startTemp,endTemp
id,1.000000,-0.131814,-0.518452,NaN,-0.508053,-0.508053,0.648869,0.698829
truck,-0.131814,1.000000,0.109887,NaN,0.231737,0.231737,0.125621,0.493346
driver,-0.518452,0.109887,1.000000,NaN,-0.028969,-0.028969,0.036081,-0.325019
shipment_distance,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
consumed_fuel,-0.508053,0.231737,-0.028969,NaN,1.000000,1.000000,-0.366547,-0.125860
fConsumption,-0.508053,0.231737,-0.028969,NaN,1.000000,1.000000,-0.366547,-0.125860
startTemp,0.648869,0.125621,0.036081,NaN,-0.366547,-0.366547,1.000000,0.776972
endTemp,0.698829,0.493346,-0.325019,NaN,-0.125860,-0.125860,0.776972,1.000000
